<h1>Extracting and Visualizing Stock Data of Tesla and GameStop</h1>



In [ ]:
# Install required libraries
!pip install yfinance
!pip install bs4
!pip install nbformat

In [4]:
# Import required libraries
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [5]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [63]:
# Function that plots the graphs
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

<h3>Using yfinance to Extract Tesla's Stock Data</h3>

In [43]:
tesla = yf.Ticker('TSLA')

In [44]:
history = tesla.history(period = "max")
tesla_data = pd.DataFrame(history)

In [ ]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

<h3> Using Webscraping to Extract Tesla's Revenue Data </h3>

In [46]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

In [47]:
soup = BeautifulSoup(html_data, 'html5lib')

In [48]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in soup.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
 
    tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)

In [49]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)

In [50]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

<h3>Using yfinance to Extract GameStop's Stock Data</h3>

In [52]:
gme = yf.Ticker('GME')

In [53]:
gme_data = pd.DataFrame(gme.history(period = "max"))

In [ ]:
gme_data.reset_index(inplace = True)
gme_data.head()

<h3> Using Webscraping to Extract GameStop's Revenue Data </h3>

In [57]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html."
html_data_2 = requests.get(url).text

In [58]:
soup2 = BeautifulSoup(html_data_2, 'html.parser')

In [59]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in soup2.find_all("tbody")[1].find_all('tr'):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
 
    gme_revenue = pd.concat([gme_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)

In [ ]:
gme_revenue.tail()

<h3>  Tesla's Stock Graph </h3>

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla')